<a href="https://colab.research.google.com/github/rohithmsr/AI-practice/blob/main/Opik/1_multi_step_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# Tracking a Multi-step LLM Chain

In this exercise, you'll track a multi-step LLM chain with Opik. You can use OpenAI or open source models via LiteLLM.

If you have multiple steps in your LLM pipeline, you can use the `track` decorator to log the traces for each step. If OpenAI is called within one of these steps, the LLM call with be associated with that corresponding step:

# Imports & Configuration

In [ ]:
%pip install opik openai --quiet

In [ ]:
from opik import track
import opik
from opik.integrations.openai import track_openai
from openai import OpenAI
import getpass
import os

os.environ["OPIK_PROJECT_NAME"] = "Multi-step-Chain-Demo"

In [ ]:
# opik configs
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

In [ ]:
# openai configs
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
client = OpenAI()
openai_client = track_openai(client)

# Define First Step

In [ ]:
@track
def generate_meal(ingredient):
    prompt = f"Generate one example of a meal that can be made with {ingredient}."
    res = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

# Define Second Step

In [ ]:
@track
def generate_recipe(meal):
    prompt = f"Generate a step-by-step recipe for {meal}"
    res = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

# Call Chain

In [ ]:
@track
def generate_recipe_from_ingredient(ingredient):
    meal = generate_meal(ingredient)
    story = generate_recipe(meal)
    return story

generate_recipe_from_ingredient("garlic")

# Try with your own example!

In [ ]:
generate_recipe_from_ingredient(input("Enter an ingredient: "))